# 서울TG 방향별 전후 속도 분석
* 양방향(전후) 2km 이내

# Import

In [304]:
import pandas as pd
import numpy as np

import os
from tqdm import tqdm

import plotly.express as px

# Load Dataset

In [305]:
Conzone_info_file = 'Conzone_info_total.csv'

In [306]:
VDS_info_file = 'VDS_list_total.csv'

In [307]:
VDS_Speed_file = 'VDS_Speed_220301-220330.csv'

In [308]:
Conzone_info = pd.read_csv(Conzone_info_file, encoding = 'cp949')

In [309]:
VDS_info = pd.read_csv(VDS_info_file, encoding = 'cp949')

In [310]:
VDS_speed = pd.read_csv(VDS_Speed_file, encoding = 'cp949')

In [311]:
len(Conzone_info), len(VDS_info), len(VDS_speed)

(2370, 7840, 23337168)

# Filtering

## Date
* 2022년 3월, 공휴일이 아닌 화/수/목 데이터만 도출 : 2, 3, 8, 9, 15, 16, 17, 22, 23, 24, 29, 30, 31

In [312]:
target_dates = [2, 3, 8, 9, 15, 16, 17, 22, 23, 24, 29, 30, 31]
dates = [i + 20220300 for i in target_dates]

In [313]:
dates

[20220302,
 20220303,
 20220308,
 20220309,
 20220315,
 20220316,
 20220317,
 20220322,
 20220323,
 20220324,
 20220329,
 20220330,
 20220331]

In [314]:
VDS_speed_filtered = VDS_speed[VDS_speed['집계일자'].isin(dates)] # 원하는 날짜만 필터링

In [315]:
VDS_speed_filtered.head(3)

,집계일자,집계시분,VDS_ID,지점이정,차로유형구분코드,평균속도,Unnamed: 6
6548,20220302,315,0170VDE00002,2.0,1,104.47,NaN
6549,20220302,315,0170VDE00003,3.0,1,109.83,NaN
6550,20220302,315,0170VDE00004,4.0,1,99.74,NaN


## Conzone
* 원하는 콘존의 VDS만 추출

In [316]:
Conzones = ['판교IC-서울TG', '서울TG-신갈JC', '신갈JC-서울TG', '서울TG-판교IC']

In [317]:
Conzone_info_filtered = Conzone_info[Conzone_info['콘존명'].isin(Conzones)]

In [318]:
Conzone_list_filtered = list(Conzone_info_filtered['콘존ID'].unique())

In [319]:
Conzone_info_filtered

,Unnamed: 0,콘존ID,콘존길이,콘존명,노선번호,도로명,도로단축명,도로표출명,기점종점방향구분코드,시점명,...,시작노드ID,종료노드ID,시작노드_도로이정,종료노드_도로이정,차로수,버스전용차로유무,도로등급구분코드,제한속도,소통원활기준속도,정체기준속도
97,97,0010CZE460,8340.0,신갈JC-서울TG,10,경부선,경부선,경부,E,부산,...,309,280,394.96,403.30,5.0,1,101,110.0,80,40
98,98,0010CZE460,8340.0,신갈JC-서울TG,10,경부선,경부선,경부,E,부산,...,309,280,41.79,403.30,5.0,1,101,110.0,80,40
99,99,0010CZE470,3640.0,서울TG-판교IC,10,경부선,경부선,경부,E,부산,...,280,274,403.30,406.94,5.0,1,101,110.0,80,40
209,209,0010CZS460,8340.0,서울TG-신갈JC,10,경부선,경부선,경부,S,부산,...,280,309,403.30,394.96,5.0,1,101,110.0,80,40
210,210,0010CZS460,8340.0,서울TG-신갈JC,10,경부선,경부선,경부,S,부산,...,280,309,403.30,41.79,5.0,1,101,110.0,80,40
211,211,0010CZS470,3640.0,판교IC-서울TG,10,경부선,경부선,경부,S,부산,...,274,280,406.94,403.30,5.0,1,101,110.0,80,40


In [320]:
Conzone_list_filtered

['0010CZE460', '0010CZE470', '0010CZS460', '0010CZS470']

## VDS
* 원하는 콘존 내의 VDS만 추출

In [321]:
VDS_info.head(3)

,Unnamed: 0,Unnamed: 0.1,pageNo,numOfRows,routeNo,routeName,vdsId,shift,vdsLength,roadgradeCode,...,equipmentBelongingName,grs80x,grs80y,vdsStartShift,vdsEndShift,vdsCode,vdsName,routeSeq,directionCode,czId
0,0,0,NaN,NaN,10,경부선,0010VDE00100,1.40km,1820.00m,101,...,본선,391265.168637,297630.018235,0.20km,2.02km,1,FTMS 본선 VDS,1.0,E,0010CZE010
1,1,1,NaN,NaN,10,경부선,0010VDE00200,2.40km,830.00m,101,...,본선,391642.598589,298552.156704,2.02km,2.85km,1,FTMS 본선 VDS,2.0,E,0010CZE011
2,2,2,NaN,NaN,10,경부선,0010VDE00300,3.30km,1160.00m,101,...,본선,391873.613256,299421.826590,2.85km,4.01km,1,FTMS 본선 VDS,3.0,E,0010CZE011


In [322]:
VDS_filtered = VDS_info[VDS_info['czId'].isin(Conzone_filtered)]

In [323]:
VDS_filtered.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'pageNo', 'numOfRows', 'routeNo',
       'routeName', 'vdsId', 'shift', 'vdsLength', 'roadgradeCode',
       'roadgradeName', 'equipmentBelongingCode', 'equipmentBelongingName',
       'grs80x', 'grs80y', 'vdsStartShift', 'vdsEndShift', 'vdsCode',
       'vdsName', 'routeSeq', 'directionCode', 'czId'],
      dtype='object')

In [324]:
VDS_filtered = VDS_filtered.loc[:, ['routeNo', 'routeName', 'vdsId', 'shift', 'vdsLength', 'roadgradeCode', 'equipmentBelongingName', 'vdsStartShift', 'vdsEndShift', 'vdsCode', 'vdsName', 'directionCode', 'czId']]

In [325]:
VDS_filtered.head(3)

,routeNo,routeName,vdsId,shift,vdsLength,roadgradeCode,equipmentBelongingName,vdsStartShift,vdsEndShift,vdsCode,vdsName,directionCode,czId
488,10,경부선,0010VDE30600,395.00km,1240.00m,101,본선,394.96km,396.20km,1,FTMS 본선 VDS,E,0010CZE460
489,10,경부선,0010VDE30650,397.40km,1950.00m,101,본선,396.20km,398.15km,1,FTMS 본선 VDS,E,0010CZE460
490,10,경부선,0010VDE30700,398.90km,1150.00m,101,본선,398.15km,399.30km,1,FTMS 본선 VDS,E,0010CZE460


## VDS Location
* 서울TG이정(403) +- 2.4km 이내의 VDS만 남기기 : 401.6 ~ 405.4

In [326]:
VDS_speed_filtered = VDS_speed_filtered[(VDS_speed_filtered['지점이정'] >= 400.6) & (VDS_speed_filtered['지점이정'] <= 405.4)]

In [327]:
VDS_list_filtered = list(VDS_filtered['vdsId'])

In [328]:
VDS_list_filtered

['0010VDE30600',
 '0010VDE30650',
 '0010VDE30700',
 '0010VDE30800',
 '0010VDE30900',
 '0010VDE30910',
 '0010VDE31000',
 '0010VDE31010',
 '0010VDE31200',
 '0010VDE31300',
 '0010VDE31400',
 '0010VDS30600',
 '0010VDS30650',
 '0010VDS30700',
 '0010VDS30800',
 '0010VDS30900',
 '0010VDS30920',
 '0010VDS30930',
 '0010VDS31000',
 '0010VDS31010',
 '0010VDS31201',
 '0010VDS31301',
 '0010VDS31400']

In [329]:
VDS_speed_filtered = VDS_speed_filtered[VDS_speed_filtered['VDS_ID'].isin(VDS_list_filtered)]

In [330]:
VDS_speed_filtered.to_csv('VDS_speed_filtered.csv', encoding = 'cp949')

In [331]:
VDS_speed_filtered

,집계일자,집계시분,VDS_ID,지점이정,차로유형구분코드,평균속도,Unnamed: 6
10297,20220302,1700,0010VDE30900,401.0,2,89.08,NaN
10300,20220302,1700,0010VDE31000,402.5,2,86.91,NaN
10303,20220302,1700,0010VDE31200,404.4,2,104.54,NaN
10306,20220302,1700,0010VDE31300,405.2,2,115.64,NaN
10349,20220302,1700,0010VDS30900,401.0,2,93.66,NaN
...,...,...,...,...,...,...,...
23329805,20220331,2345,0010VDE31300,405.2,1,100.53,NaN
23330222,20220331,2345,0010VDS30900,401.0,1,97.20,NaN
23330226,20220331,2345,0010VDS31000,402.5,1,96.58,NaN
23330229,20220331,2345,0010VDS31201,404.4,1,113.29,NaN


# Average
* 화/수/목요일 평균을 내기
    * 화 : 8, 15, 22, 29
    * 수 : 2, 16, 23, 30
    * 목 : 3, 10, 17, 24, 31
* 차로유형구분코드, VDS_ID, 집계시분별 평균임

In [332]:
Tue = [8, 15, 22, 29]
Wed = [2, 16, 23, 30]
Thu = [3, 10, 17, 24, 31]

def dayfunc(x):
    if x - 20220300 in Tue:
        return 'Tue'
    elif x - 20220300 in Wed:
        return 'Wed'
    elif x - 20220300 in Thu:
        return 'Thu'
    else:
        pass

In [333]:
VDS_speed_filtered['day'] = VDS_speed_filtered['집계일자'].apply(dayfunc)

In [334]:
VDS_speed_filtered.head(3)

,집계일자,집계시분,VDS_ID,지점이정,차로유형구분코드,평균속도,Unnamed: 6,day
10297,20220302,1700,0010VDE30900,401.0,2,89.08,NaN,Wed
10300,20220302,1700,0010VDE31000,402.5,2,86.91,NaN,Wed
10303,20220302,1700,0010VDE31200,404.4,2,104.54,NaN,Wed


In [335]:
VDS_speed_pivoted = pd.pivot_table(VDS_speed_filtered,
                                   index = ['day', 'VDS_ID', '지점이정', '차로유형구분코드', '집계시분'],
                                   values = '평균속도', aggfunc = 'mean').reset_index()

In [336]:
def direction(VDS):
    """VDS ID로 방향을 식별하는 함수"""
    direct = VDS[6]
    
    return direct

In [337]:
VDS_speed_filtered['direction'] = VDS_speed_filtered['VDS_ID'].apply(direction)

In [338]:
VDS_speed_pivoted.head(3)

,day,VDS_ID,지점이정,차로유형구분코드,집계시분,평균속도
0,Thu,0010VDE30900,401.0,1,0,95.2550
1,Thu,0010VDE30900,401.0,1,15,97.0025
2,Thu,0010VDE30900,401.0,1,30,95.0750


In [339]:
VDS_speed_pivoted.to_csv('VDS_speed_pivoted.csv', encoding = 'cp949')

# Graph
* 서울TG 이정 : 403
* 방향 :
    * E : 서울방향
    * S : 부산방향   
* 필요 데이터
    * 전체 요일 평균(화, 수, 목 각각!!) 
    * 영업소 직전 VDS 평균 : 화, 수, 목 각각 평균
    * 영업소 직후 VDS 평균 : 화, 수, 목 각각!!

## 전체 평균
* 서울방향(E) : 0010VDE30900(401), 0010VDE31000(402.5), 0010VDE31200(404.4), 0010VDE31300(405.2)
* 부산방향(S) : 0010VDS30900(401), 0010VDS31000(402.5), 0010VDS31201(404.4), 0010VDS31301(405.2)

In [340]:
VDS_speed_pivoted['direction'] = VDS_speed_pivoted['VDS_ID'].apply(direction)

In [341]:
VDS_speed_pivoted_total = pd.pivot_table(VDS_speed_pivoted, index = ['direction', 'day', '차로유형구분코드', '집계시분'], values = '평균속도', aggfunc = 'mean').reset_index()

In [342]:
VDS_speed_pivoted_total.head(3)

,direction,day,차로유형구분코드,집계시분,평균속도
0,E,Thu,1,0,101.075625
1,E,Thu,1,15,102.155000
2,E,Thu,1,30,99.906875


In [343]:
direction = ['E', 'S']
lanecode = [1, 2]

save_html_dir = 'VDS_speed_total_html'

if not os.path.exists(save_html_dir):
    os.makedirs(save_html_dir)
    
for di in direction:
    for la in lanecode:
        fig = px.line(VDS_speed_pivoted_total[(VDS_speed_pivoted_total['direction'] == di) & 
                                                  #(VDS_speed_pivoted_total['day'] == da) & 
                                                (VDS_speed_pivoted_total['차로유형구분코드'] == la)],
                        x = '집계시분', y = '평균속도', color = 'day',
                        title = f'집계시분별 전체구간 평균속도 : 방향 {di}, 차로유형구분코드 {la}')
            
        save_html_file = f'VDS_Speed_total_{di}_{la}.html'
        save_html_path = os.path.join(save_html_dir, save_html_file)
            
        fig.write_html(save_html_path)

In [344]:
pd.pivot_table(VDS_speed_pivoted_total, index = ['집계시분'], columns = ['direction', '차로유형구분코드', 'day'], values = '평균속도').to_csv('FIN_VDS_speed_total.csv', encoding = 'cp949')

## 영업소 직전 VDS 평균
* 서울방향(E) : 0010VDE30900(401), 0010VDE31000(402.5)
* 부산방향(S) : 0010VDS31201(404.4), 0010VDS31301(405.2)

In [345]:
VDS_before_TG_list = ['0010VDE30900', '0010VDE31000', '0010VDS31201', '0010VDS31301']

VDS_speed_before_TG = VDS_speed_pivoted[VDS_speed_pivoted['VDS_ID'].isin(VDS_before_TG_list)]

In [346]:
VDS_speed_before_TG = pd.pivot_table(VDS_speed_before_TG, index = ['direction', 'day', '차로유형구분코드', '집계시분'], values = '평균속도', aggfunc = 'mean').reset_index()

In [347]:
VDS_speed_before_TG.head(3)

,direction,day,차로유형구분코드,집계시분,평균속도
0,E,Thu,1,0,95.58750
1,E,Thu,1,15,97.11250
2,E,Thu,1,30,95.02875


In [348]:
direction = ['E', 'S']
lanecode = [1, 2]

save_html_dir = 'VDS_speed_before_TG_html'

if not os.path.exists(save_html_dir):
    os.makedirs(save_html_dir)
    
for di in direction:
    for la in lanecode:
        fig = px.line(VDS_speed_before_TG[(VDS_speed_before_TG['direction'] == di) & 
                                          (VDS_speed_before_TG['차로유형구분코드'] == la)],
                        x = '집계시분', y = '평균속도', color = 'day',
                        title = f'집계시분별 서울TG 전 평균속도 : 방향 {di}, 차로유형구분코드 {la}')
            
        save_html_file = f'VDS_Speed_before_TG_{di}_{la}.html'
        save_html_path = os.path.join(save_html_dir, save_html_file)
            
        fig.write_html(save_html_path)

In [349]:
pd.pivot_table(VDS_speed_before_TG, index = ['집계시분'], columns = ['direction', '차로유형구분코드', 'day'], values = '평균속도').to_csv('FIN_VDS_speed_before_TG.csv', encoding = 'cp949')

## 영업소 직후 VDS 평균
* 서울방향(E) : 0010VDE31200(404.4), 0010VDE31300(405.2)
* 부산방향(S) : 0010VDS30900(401), 0010VDS31000(402.5)

In [361]:
VDS_after_TG_list = ['0010VDE31200', '0010VDE31300', '0010VDS30900', '0010VDS31000']

VDS_speed_after_TG = VDS_speed_pivoted[VDS_speed_pivoted['VDS_ID'].isin(VDS_after_TG_list)]

In [362]:
VDS_speed_pivoted[VDS_speed_pivoted['VDS_ID'].isin(VDS_after_TG_list)]

,day,VDS_ID,지점이정,차로유형구분코드,집계시분,평균속도,direction
304,Thu,0010VDE31200,404.4,1,0,112.0250,E
305,Thu,0010VDE31200,404.4,1,15,113.6525,E
306,Thu,0010VDE31200,404.4,1,30,110.4300,E
307,Thu,0010VDE31200,404.4,1,45,111.5400,E
308,Thu,0010VDE31200,404.4,1,100,108.9800,E
...,...,...,...,...,...,...,...
3339,Wed,0010VDS31000,402.5,2,1945,93.1475,S
3340,Wed,0010VDS31000,402.5,2,2000,92.0025,S
3341,Wed,0010VDS31000,402.5,2,2015,94.0650,S
3342,Wed,0010VDS31000,402.5,2,2030,94.4800,S


In [363]:
VDS_speed_after_TG = pd.pivot_table(VDS_speed_after_TG, index = ['direction', 'day', '차로유형구분코드', '집계시분'], values = '평균속도', aggfunc = 'mean').reset_index()

In [364]:
VDS_speed_after_TG

,direction,day,차로유형구분코드,집계시분,평균속도
0,E,Thu,1,0,106.56375
1,E,Thu,1,15,107.19750
2,E,Thu,1,30,104.78500
3,E,Thu,1,45,105.24375
4,E,Thu,1,100,102.92500
...,...,...,...,...,...
907,S,Wed,2,1945,92.44750
908,S,Wed,2,2000,91.74875
909,S,Wed,2,2015,93.25125
910,S,Wed,2,2030,93.82500


In [365]:
direction = ['E', 'S']
lanecode = [1, 2]

save_html_dir = 'VDS_speed_after_TG_html'

if not os.path.exists(save_html_dir):
    os.makedirs(save_html_dir)
    
for di in direction:
    for la in lanecode:
        fig = px.line(VDS_speed_after_TG[(VDS_speed_after_TG['direction'] == di) & 
                                        (VDS_speed_after_TG['차로유형구분코드'] == la)],
                        x = '집계시분', y = '평균속도', color = 'day',
                        title = f'집계시분별 서울TG 전 평균속도 : 방향 {di}, 차로유형구분코드 {la}')
            
        save_html_file = f'VDS_Speed_after_TG_{di}_{la}.html'
        save_html_path = os.path.join(save_html_dir, save_html_file)
            
        fig.write_html(save_html_path)

In [366]:
pd.pivot_table(VDS_speed_after_TG, index = ['집계시분'], columns = ['direction', '차로유형구분코드', 'day'], values = '평균속도').to_csv('FIN_VDS_speed_after_TG.csv', encoding = 'cp949')